Utilisation du kernel PySpark

In [1]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1696687097932_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1696687097932_0001,pyspark,idle,Link,Link,None,✔


In [3]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import sys
print(sys.version)
print(sys.executable)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3.7.16 (default, Aug 30 2023, 20:37:53) 
[GCC 7.3.1 20180712 (Red Hat 7.3.1-15)]
/usr/bin/python3

In [7]:
PATH = 's3://bucket-projet-8'
PATH_Data = PATH+'/test'
PATH_Result = PATH+'/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://bucket-projet-8
PATH_Data:   s3://bucket-projet-8/test
PATH_Result: s3://bucket-projet-8/Results

In [8]:
#Chargement des images au format binaire (uniquement les jpg)
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+--------------------------------------------------+----------+
|path                                              |label     |
+--------------------------------------------------+----------+
|s3://bucket-projet-8/test/Watermelon/r_106_100.jpg|Watermelon|
|s3://bucket-projet-8/test/Watermelon/r_109_100.jpg|Watermelon|
|s3://bucket-projet-8/test/Watermelon/r_108_100.jpg|Watermelon|
|s3://bucket-projet-8/test/Watermelon/r_107_100.jpg|Watermelon|
|s3://bucket-projet-8/test/Watermelon/r_95_100.jpg |Watermelon|
+--------------------------------------------------+----------+
only showing top 5 rows

None

## Model Preparation

In [10]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14540800/14536120 [==============================] - 1s 0us/step

In [11]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [13]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    #Garantit à l'ensemble des noeuds les mêmes poids pré-entrainés
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Chargement des images et featurisation via Pandas Udf

In [15]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt1/yarn/usercache/livy/appcache/application_1696687097932_0001/container_1696687097932_0001_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

## Extraction de features

In [16]:
#spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
features_df = images.repartition(20).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import VectorUDT, Vectors
from pyspark.sql.functions import udf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Fonction UDF pour la conversion
def array_to_vector(arr):
    return Vectors.dense(arr)

# Enregistrement de la fonction UDF
array_to_vector_udf = udf(array_to_vector, VectorUDT())

# Avant la réduction de dimension PCA, convertissez la colonne "features"
features = features_df.withColumn("features", array_to_vector_udf("features"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# Création d'une instance PCA en spécifiant le nombre de composantes principales souhaité
pca = PCA(k=20, inputCol="features", outputCol="pcaFeatures")

# Application de la transformation PCA sur features_df
pcaModel = pca.fit(features)
pcaResult = pcaModel.transform(features)

# Affichage du résultat
pcaResult.select("label", "pcaFeatures").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------------------+
|         label|         pcaFeatures|
+--------------+--------------------+
|    Watermelon|[-2.4912058827613...|
|    Watermelon|[-2.8194005345133...|
|   Cauliflower|[-4.2470413505814...|
|   Cauliflower|[-4.3164372787506...|
|     Raspberry|[-2.0753602575040...|
|   Cauliflower|[-3.3965369586463...|
| Cucumber Ripe|[-1.7941493046817...|
|   Cauliflower|[-6.8803516857844...|
|   Cauliflower|[-5.3257854066306...|
| Cucumber Ripe|[0.19829398022476...|
|   Onion White|[-1.2629180496028...|
|        Lychee|[-3.9788973013690...|
|      Pear Red|[2.27132280372063...|
|Apple Golden 1|[-4.9480641964554...|
|      Pear Red|[4.01235663603525...|
|     Mandarine|[0.82659010548122...|
|  Pear Forelle|[1.32351293732252...|
| Pepper Orange|[2.57306709799909...|
|    Strawberry|[0.24450846020346...|
|  Cantaloupe 1|[0.58339728568872...|
+--------------+--------------------+
only showing top 20 rows

In [36]:
features_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+--------------------+
|                path|         label|            features|
+--------------------+--------------+--------------------+
|s3://bucket-proje...|Pineapple Mini|[0.0, 4.9140387, ...|
|s3://bucket-proje...|    Watermelon|[0.6964483, 0.001...|
|s3://bucket-proje...|     Raspberry|[0.015883103, 0.4...|
|s3://bucket-proje...|   Cauliflower|[0.0038410448, 1....|
|s3://bucket-proje...|     Pineapple|[0.0, 4.228983, 0...|
|s3://bucket-proje...|   Cauliflower|[0.0, 1.2145613, ...|
|s3://bucket-proje...|   Cauliflower|[0.0, 0.14606494,...|
|s3://bucket-proje...|Pineapple Mini|[0.02307794, 4.44...|
|s3://bucket-proje...|   Cauliflower|[0.0, 0.91174525,...|
|s3://bucket-proje...|Apple Golden 1|[0.0, 0.0, 0.7517...|
|s3://bucket-proje...|      Pear Red|[5.3999397E-6, 0....|
|s3://bucket-proje...|   Onion White|[0.10229113, 1.05...|
|s3://bucket-proje...|      Pear Red|[0.031456675, 0.0...|
|s3://bucket-proje...|        Lychee|[1.0082625, 2.292..

In [22]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bucket-projet-8/Results

In [23]:
#Enregistrement au format parquet
features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
#Enregistrement ACP au format parquet
pcaResult.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interrupted by user


In [44]:
# Enregistrement au format Parquet (ajout au dossier existant)
features_df.write.mode("append").parquet(PATH_Result)

# Enregistrement ACP au format Parquet (ajout au dossier existant)
pcaResult.write.mode("append").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Chargment des données enregistrées et validation du résultat

In [28]:
#Chargement des données "résultats" dans un dataframe
df = pd.read_parquet(PATH_Result, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
module 'charset_normalizer' has no attribute 'md__mypyc'
Traceback (most recent call last):
  File "/usr/local/lib64/python3.7/site-packages/pandas/io/parquet.py", line 500, in read_parquet
    **kwargs,
  File "/usr/local/lib64/python3.7/site-packages/pandas/io/parquet.py", line 236, in read
    mode="rb",
  File "/usr/local/lib64/python3.7/site-packages/pandas/io/parquet.py", line 84, in _get_path_or_handle
    path_or_handle, **(storage_options or {})
  File "/usr/local/lib/python3.7/site-packages/fsspec/core.py", line 353, in url_to_fs
    chain = _un_chain(url, kwargs)
  File "/usr/local/lib/python3.7/site-packages/fsspec/core.py", line 315, in _un_chain
    cls = get_filesystem_class(protocol)
  File "/usr/local/lib/python3.7/site-packages/fsspec/registry.py", line 211, in get_filesystem_class
    register_implementation(protocol, _import_class(bit["class"]))
  File "/usr/local/lib/python3.7/site-packages/fsspec/registry.py", line 234, in _import_class
 

In [29]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df' is not defined
Traceback (most recent call last):
NameError: name 'df' is not defined



In [23]:
df.loc[0,'features'].shape

(1280,)

23/10/03 03:31:56 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 28671303 ms exceeds timeout 120000 ms
23/10/03 03:31:56 WARN SparkContext: Killing executors is not supported by current scheduler.
23/10/03 03:32:02 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at